In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

In [ ]:
from keras.preprocessing import image        

In [ ]:
from keras.models import load_model
import h5py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle

filename = '/content/drive/MyDrive/Broner/InceptionResnet/Ml/finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
m = load_model('/content/drive/MyDrive/Broner/model.h5')
m1 = load_model('/content/drive/MyDrive/Broner/InceptionResnetv2/0.hdf5')
m2 = load_model('/content/drive/MyDrive/Broner/InceptionResnetv2/1.hdf5')
m3 = load_model('/content/drive/MyDrive/Broner/InceptionResnetv2/2.hdf5')
m4 = load_model('/content/drive/MyDrive/Broner/InceptionResnetv2/3.hdf5')
m5 = load_model('/content/drive/MyDrive/Broner/InceptionResnetv2/4.hdf5')

In [ ]:
def check_value(var):
  
  if var>0.5:
    return 1
  return 0
def new_answer(img):
  arr = np.empty(5, dtype=int)
  # img = image.load_img(path,target_size=(320,320))
  img_tensor = image.img_to_array(img)
  img_tensor = np.expand_dims(img_tensor,axis = 0)
  img_tensor /= 255
  img_tensor[:,:,0] = (img_tensor[:,:,0]-0.485)/0.229
  img_tensor[:,:,1] = (img_tensor[:,:,1]-0.456)/0.224
  img_tensor[:,:,2] = (img_tensor[:,:,2]-0.406)/0.225
  ans1 = m1.predict(img_tensor)
  arr[0] = check_value(ans1)
  ans2 = m2.predict(img_tensor) 
  arr[1] = check_value(ans2)
  ans3 = m3.predict(img_tensor)
  arr[2] = check_value(ans3)
  ans4 = m4.predict(img_tensor)
  arr[3] = check_value(ans4)
  ans5 = m5.predict(img_tensor)
  arr[4] = check_value(ans5)
  arr = arr.reshape(1,-1)
  ans = m.predict(img_tensor)
  a = np.argmax(ans)
  if a == 0:
    bone = 'Shoulder'
  elif a == 1:
    bone = 'Humerus' 
  else:
    bone = 'Forearm'
  # print(loaded_model.predict(arr))
  return loaded_model.predict(arr),bone

In [ ]:
!pip install flask-ngrok

In [ ]:
import numpy
import scipy.misc
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Model
from keras.preprocessing import image 
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
def grad_cam(model, img_tensor,
             layer_name="conv_7b_ac", label_name=None,
             category_id=None):
    """Get a heatmap by Grad-CAM.
    Args:
        model: A model object, build from tf.keras 2.X.
        img: An image ndarray.
        layer_name: A string, layer name in model.
        label_name: A list,
            show the label name by assign this argument,
            it should be a list of all label names.
        category_id: An integer, index of the class.
            Default is the category with the highest score in the prediction.
    Return:
        A heatmap ndarray(without color).
    """
    heatmap_model = Model([model.inputs], [model.get_layer(layer_name).output , model.output])
    with tf.GradientTape() as gtape:
        conv_output, predictions = heatmap_model(img_tensor)
        if category_id == None:
            if predictions[0]<0.5:
              category_id = 0
            else:
              category_id = 1
        if label_name:
            print(label_name[category_id])
        output = predictions[:,0]
        grads = gtape.gradient(output, conv_output)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    max_heat = np.max(heatmap)
    if max_heat == 0:
        max_heat = 1e-10
    heatmap /= max_heat

    return np.squeeze(heatmap)

def grad_cam_plus(model, img_tensor,
                  layer_name="conv_7b_ac", label_name=None,
                  category_id=None):
    """Get a heatmap by Grad-CAM.
    Args:
        model: A model object, build from tf.keras 2.X.
        img: An image ndarray.
        layer_name: A string, layer name in model.
        label_name: A list,
            show the label name by assign this argument,
            it should be a list of all label names.
        category_id: An integer, index of the class.
            Default is the category with the highest score in the prediction.
    Return:
        A heatmap ndarray(without color).
    """
    conv_layer = model.get_layer(layer_name)
    heatmap_model = Model([model.inputs], [conv_layer.output, model.output])

    with tf.GradientTape() as gtape1:
        with tf.GradientTape() as gtape2:
            with tf.GradientTape() as gtape3:
                conv_output, predictions = heatmap_model(img_tensor)
                if category_id==None:
                    if predictions[0]<0.5:
                      category_id = 0
                    else:
                      category_id = 1
                if label_name:
                    print(label_name[category_id])
                output = predictions[:,0]
                conv_first_grad = gtape3.gradient(output, conv_output)
            conv_second_grad = gtape2.gradient(conv_first_grad, conv_output)
        conv_third_grad = gtape1.gradient(conv_second_grad, conv_output)

    global_sum = np.sum(conv_output, axis=(0, 1, 2))

    alpha_num = conv_second_grad[0]
    alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, 1e-10)
    
    alphas = alpha_num/alpha_denom
    alpha_normalization_constant = np.sum(alphas, axis=(0,1))
    alphas /= alpha_normalization_constant

    weights = np.maximum(conv_first_grad[0], 0.0)

    deep_linearization_weights = np.sum(weights*alphas, axis=(0,1))
    grad_CAM_map = np.sum(deep_linearization_weights*conv_output[0], axis=2)

    heatmap = np.maximum(grad_CAM_map, 0)
    max_heat = np.max(heatmap)
    if max_heat == 0:
        max_heat = 1e-10
    heatmap /= max_heat

    return heatmap

In [ ]:
def find_target_layer(model):
		# attempt to find the final convolutional layer in the network
		# by looping over the layers of the network in reverse order
		for layer in reversed(model.layers):
			# check to see if the layer has a 4D output
			if len(layer.output_shape) == 4:
				return layer.name
		# otherwise, we could not find a 4D layer so the GradCAM
		# algorithm cannot be applied
		raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")

In [ ]:
def show_imgwithheat(img_path, heatmap, alpha=0.4, return_array=True):
    """Show the image with heatmap.
    Args:
        img_path: string.
        heatmap:  image array, get it by calling grad_cam().
        alpha:    float, transparency of heatmap.
        return_array: bool, return a superimposed image array or not.
    Return:
        None or image array.
    """
    img = cv2.imread(img_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = (heatmap*255).astype("uint8")
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + img
    superimposed_img = np.clip(superimposed_img, 0, 255).astype("uint8")
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

    imgwithheat = Image.fromarray(superimposed_img) 
    # imgwithheat.save('/content/drive/MyDrive/static/heat.jpg') 
    # plt.imshow(imgwithheat)
    # return imgwithheat
    if return_array:
        return superimposed_img

In [ ]:
def preprocess(img):
  resized = cv2.resize(img, (320,320))
  img_tensor = image.img_to_array(resized)
  img_tensor = np.expand_dims(img_tensor,axis = 0)
  img_tensor /= 255
  img_tensor[:,:,0] = (img_tensor[:,:,0]-0.485)/0.229
  img_tensor[:,:,1] = (img_tensor[:,:,1]-0.456)/0.224
  img_tensor[:,:,2] = (img_tensor[:,:,2]-0.406)/0.225
  return img_tensor

In [ ]:
img_path = '/content/drive/MyDrive/Broner/MURA-v1.1/train/XR_HUMERUS/patient00051/study1_positive/image1.png'
img = cv2.imread(img_path)
img = preprocess(img)
heatmap_plus = grad_cam_plus(m4, img , label_name=['Negative' , 'Positive'])
show_imgwithheat(img_path, heatmap_plus)

Positive


array([[[102,  97,   0],
        [102,  97,   0],
        [102,  97,   0],
        ...,
        [ 67,  67, 161],
        [ 68,  68, 162],
        [ 67,  67, 161]],

       [[102,  97,   0],
        [102,  97,   0],
        [102,  97,   0],
        ...,
        [ 68,  68, 162],
        [ 67,  67, 161],
        [ 67,  67, 161]],

       [[102,  97,   0],
        [102,  97,   0],
        [102,  97,   0],
        ...,
        [ 66,  66, 160],
        [ 67,  67, 161],
        [ 67,  67, 161]],

       ...,

       [[151, 166,  79],
        [152, 167,  80],
        [153, 168,  81],
        ...,
        [  0,   0, 100],
        [  0,   0, 100],
        [  0,   0, 100]],

       [[153, 168,  81],
        [153, 168,  81],
        [151, 166,  79],
        ...,
        [  0,   0, 100],
        [  0,   0, 100],
        [  0,   0, 100]],

       [[152, 167,  80],
        [151, 166,  79],
        [152, 167,  80],
        ...,
        [  0,   0, 100],
        [  0,   0, 100],
        [  0,   0, 100]]

In [ ]:
from flask import Flask,render_template,request
from flask_ngrok import run_with_ngrok
import cv2,base64,re
import werkzeug
import keras.models
app = Flask(__name__,template_folder='/content/drive/MyDrive/templates',static_folder='/content/drive/MyDrive/static')
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0
run_with_ngrok(app)   
# app.debug = True
@app.route("/")
def home():
    return render_template('index.html',link = 'left.jpg')


@app.route("/image",methods=['POST','GET'])
def answers():
  link = 'left.jpg'
  if request.method == 'POST':
    imagefile = request.files['img']
    filename = 'temp.jpg'
    print("\nReceived image File name : " + imagefile.filename)
    imagefile.save(filename)
    
    img = cv2.imread(filename)
    resized = cv2.resize(img, (320,320))
    heat = preprocess(resized)
    heatmap_plus = grad_cam(m4,heat , label_name=['Negative' , 'Positive'])
    arr = show_imgwithheat('temp.jpg', heatmap_plus) 
    # arr.save('/content/drive/MyDrive/static/heat.jpg') 
    cv2.imwrite('/content/drive/MyDrive/static/heat.jpg',arr)
    cv2.imwrite('heat.jpg',arr)
    result,bone = new_answer(resized)
    if result[0] == 0:
      rs = 'negative'
    else:
      rs = 'positive'
    return render_template('index.html',results = rs,bone = bone,link = 'heat.jpg')
  else:
    return render_template('index.html',link = link)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c34c1f35d338.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/Apr/2021 11:01:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2021 11:01:02] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2021 11:01:02] "GET /static/left.jpg HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2021 11:01:04] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Apr/2021 11:01:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2021 11:01:06] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2021 11:01:06] "GET /static/left.jpg HTTP/1.1" 200 -



Received image File name : image1.png
Positive


127.0.0.1 - - [30/Apr/2021 11:01:09] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2021 11:01:10] "GET /static/left.jpg HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2021 11:01:10] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2021 11:01:14] "POST /image HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2021 11:01:14] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [30/Apr/2021 11:01:14] "GET /static/heat.jpg HTTP/1.1" 200 -
